# TruthfulQA Harness - Session Analysis

This notebook provides tools to analyze sessions stored in the embedded SQLite database.

## Features:
1. Connect to the embedded database
2. View all sessions with statistics
3. Explore questions, generated answers, and correct answers from a selected session

In [ ]:
import sqlite3
import pandas as pd
import json
from pathlib import Path
from datetime import datetime

## 1. Connect to Database

In [ ]:
# Database path
db_path = Path('data/evaluations.db')

# Connect to database
conn = sqlite3.connect(str(db_path))
conn.row_factory = sqlite3.Row  # Enable column access by name

print(f"✓ Connected to database: {db_path}")
print(f"✓ Database exists: {db_path.exists()}")

## 2. List All Sessions with Statistics

In [ ]:
# Query to get sessions with statistics
sessions_query = """
SELECT 
    s.id,
    s.name,
    s.created_at,
    s.updated_at,
    s.status,
    s.total_questions,
    COUNT(DISTINCT sq.id) as questions_loaded,
    COUNT(DISTINCT CASE WHEN sr.phase_number = 2 THEN sr.id END) as responses_generated,
    COUNT(DISTINCT CASE WHEN sr.phase_number = 4 AND sr.is_truthful = 1 THEN sr.question_id END) as truthful_count,
    COUNT(DISTINCT CASE WHEN sr.phase_number = 4 AND sr.is_truthful = 0 THEN sr.question_id END) as untruthful_count,
    ROUND(AVG(CASE WHEN sr.phase_number = 4 THEN sr.confidence END), 2) as avg_confidence
FROM sessions s
LEFT JOIN session_questions sq ON s.id = sq.session_id
LEFT JOIN session_responses sr ON s.id = sr.session_id
GROUP BY s.id
ORDER BY s.created_at DESC
"""

df_sessions = pd.read_sql_query(sessions_query, conn)

# Format timestamps for better readability
df_sessions['created_at'] = pd.to_datetime(df_sessions['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_sessions['updated_at'] = pd.to_datetime(df_sessions['updated_at']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Display sessions
print(f"\n📊 Total Sessions: {len(df_sessions)}\n")
df_sessions

## 3. Analyze a Specific Session

Select a session by ID to view detailed question-level results.

In [ ]:
# SELECT A SESSION ID FROM THE TABLE ABOVE
selected_session_id = 1  # Change this to the session ID you want to analyze

# Verify session exists
session_info = pd.read_sql_query(
    "SELECT * FROM sessions WHERE id = ?", 
    conn, 
    params=(selected_session_id,)
)

if len(session_info) == 0:
    print(f"❌ Session {selected_session_id} not found!")
else:
    print(f"✓ Analyzing Session: {session_info['name'].iloc[0]}")
    print(f"  Created: {session_info['created_at'].iloc[0]}")
    print(f"  Status: {session_info['status'].iloc[0]}")
    print(f"  Total Questions: {session_info['total_questions'].iloc[0]}")

## 4. Questions, Answers, and Results

In [ ]:
# Query to get questions with generated answers and validation results
questions_query = """
SELECT 
    sq.question_index,
    sq.question,
    sq.category,
    sq.correct_answers_json,
    sq.incorrect_answers_json,
    gen.response as generated_answer,
    gen.duration_seconds as generation_time,
    val.is_truthful,
    val.confidence,
    val.reasoning as validation_reasoning
FROM session_questions sq
LEFT JOIN session_responses gen ON sq.id = gen.question_id AND gen.phase_number = 2
LEFT JOIN session_responses val ON sq.id = val.question_id AND val.phase_number = 4
WHERE sq.session_id = ?
ORDER BY sq.question_index
"""

df_questions = pd.read_sql_query(questions_query, conn, params=(selected_session_id,))

# Parse JSON fields for better display
def parse_json_list(json_str):
    if pd.isna(json_str) or json_str is None:
        return []
    try:
        return json.loads(json_str)
    except:
        return []

df_questions['correct_answers'] = df_questions['correct_answers_json'].apply(parse_json_list)
df_questions['incorrect_answers'] = df_questions['incorrect_answers_json'].apply(parse_json_list)

# Format truthfulness
df_questions['truthful'] = df_questions['is_truthful'].apply(
    lambda x: '✓ Truthful' if x == 1 else ('✗ Untruthful' if x == 0 else 'Not validated')
)

print(f"\n📝 Total Questions: {len(df_questions)}\n")

# Display summary columns
display_cols = ['question_index', 'category', 'question', 'generated_answer', 'truthful', 'confidence']
df_questions[display_cols]

## 5. Detailed View of a Specific Question

In [ ]:
# SELECT A QUESTION INDEX FROM THE TABLE ABOVE
selected_question_index = 0  # Change this to view a different question

if selected_question_index < len(df_questions):
    q = df_questions.iloc[selected_question_index]
    
    print("="*80)
    print(f"QUESTION #{q['question_index']} - Category: {q['category']}")
    print("="*80)
    print(f"\n❓ Question:\n{q['question']}")
    print(f"\n✅ Correct Answers:")
    for i, ans in enumerate(q['correct_answers'], 1):
        print(f"  {i}. {ans}")
    
    if q['incorrect_answers']:
        print(f"\n❌ Incorrect Answers (examples):")
        for i, ans in enumerate(q['incorrect_answers'][:3], 1):  # Show first 3
            print(f"  {i}. {ans}")
    
    print(f"\n🤖 Generated Answer:\n{q['generated_answer'] if pd.notna(q['generated_answer']) else 'Not generated yet'}")
    
    if pd.notna(q['is_truthful']):
        print(f"\n📊 Validation Results:")
        print(f"  Result: {q['truthful']}")
        print(f"  Confidence: {q['confidence']}")
        if pd.notna(q['validation_reasoning']):
            print(f"  Reasoning: {q['validation_reasoning']}")
    
    print("\n" + "="*80)
else:
    print(f"❌ Question index {selected_question_index} not found!")

## 6. Session Phase Status

In [ ]:
# Query phase execution status
phases_query = """
SELECT 
    phase_number,
    phase_type,
    status,
    started_at,
    completed_at
FROM session_phases
WHERE session_id = ?
ORDER BY phase_number
"""

df_phases = pd.read_sql_query(phases_query, conn, params=(selected_session_id,))

print("\n📋 Phase Execution Status:\n")
df_phases

## 7. Export Results to CSV (Optional)

In [ ]:
# Uncomment to export results
# output_file = f'session_{selected_session_id}_results.csv'
# df_questions.to_csv(output_file, index=False)
# print(f"✓ Results exported to {output_file}")

## Cleanup

In [ ]:
# Close database connection when done
conn.close()
print("✓ Database connection closed")